#  Use `MlflowClient` to do model Tracking and Registry

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

The documentation is very clean, there are alot of functions that are not shown in the code. 
https://mlflow.org/docs/latest/python_api/mlflow.client.html


In [34]:
from mlflow.client import MlflowClient

# Create client of MLflow Tracking Server and Registry Server.
'''
To instantiate it we need to pass a tracking URI and/or a registry URI
Previously, we set model tracking url using 
mlflow.set_tracking_uri("sqlite:///mlflow.db") 
So we set our mlflowclient to this url to get access to those models we stored in this url 
'''


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

## Model Tracking
Interactive with MLflow Tracking Server to manage experiments and runs.

### Work with experiments - search, create, rename, delete and restore 

In [28]:

# a. get all the available experiments
print(client.search_experiments())
# b. create a new experiment
# client.create_experiment(name="my-cool-experiment") # will output experiment_id of the new experiment
# c. delete an experiment 
# client.delete_experiment(experiment_id = '2') 

[<Experiment: artifact_location='/Users/yujingma/Documents/Learning/MLOPS/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1685479041988, experiment_id='1', last_update_time=1685479041988, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>, <Experiment: artifact_location='/Users/yujingma/Documents/Learning/MLOPS/mlops-zoomcamp/02-experiment-tracking/mlruns/0', creation_time=1685479041960, experiment_id='0', last_update_time=1685479041960, lifecycle_stage='active', name='Default', tags={}>]


### Work with runs within specific experiment

In [29]:

from mlflow.entities import ViewType
'''Use ViewType to define what types of runs
we want to search. 
ACTIVE_ONLY, DELETED_ONLY, or ALL runs 
'''
runs = client.search_runs(
    experiment_ids= '1',
    filter_string= "", # empty search query
    run_view_type= ViewType.ACTIVE_ONLY, # only want to search for active runs
    max_results= 5,
    order_by=['metrics.rmse']   
)

## Or we can create filters 
runs = client.search_runs(
    experiment_ids='1',
    filter_string= 'metrics.rmse < 6.8',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by = ['metrics.rmse ASC']
)

In [33]:
for run in runs:
    print(f"run id = {run.info.run_id}, \
          lifecycle_stage: {run.info.lifecycle_stage}, \
          rmse:{run.data.metrics['rmse']:.4f}")

run id = 30f8bee56de544a0945d74fb23f3183f,           lifecycle_stage: active,           rmse:6.3632
run id = 9d5d2b92637449dabd4613e694aa85af,           lifecycle_stage: active,           rmse:6.4018
run id = cffa3bba1f2f442495bbea6e0bc9fd47,           lifecycle_stage: active,           rmse:6.4097
run id = 065d22868622429ea049621775a5cd01,           lifecycle_stage: active,           rmse:6.4161
run id = 4504246f29f2471c9e30a8edb7048d29,           lifecycle_stage: active,           rmse:6.5331


## Model Registry

Important Notes: **Model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage**, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.


In this section We will use the `MlflowClient` instance to:

In [38]:
import mlflow
# We need to set tracking uri to make sure we are not looking at the local folder
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

1. Register a new version for the experiment `nyc-taxi-regressor`

In [44]:
# register a specific run
'''
Since we already have Registered model 'nyc-taxi-regressor' registered. 
The code will make the run as a new version of this model...
'''
run_id = "30f8bee56de544a0945d74fb23f3183f"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")



Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/06/07 10:52:59 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 4
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1686149579432, current_stage='None', description=None, last_updated_timestamp=1686149579432, name='nyc-taxi-regressor', run_id='30f8bee56de544a0945d74fb23f3183f', run_link=None, source='/Users/yujingma/Documents/Learning/MLOPS/mlops-zoomcamp/02-experiment-tracking/mlruns/1/30f8bee56de544a0945d74fb23f3183f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>


2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version was created.


In [45]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 4, stage: None


3. Update the model version stage to "Staging" 

In [46]:
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1686149579432, current_stage='Staging', description=None, last_updated_timestamp=1686149674077, name='nyc-taxi-regressor', run_id='30f8bee56de544a0945d74fb23f3183f', run_link=None, source='/Users/yujingma/Documents/Learning/MLOPS/mlops-zoomcamp/02-experiment-tracking/mlruns/1/30f8bee56de544a0945d74fb23f3183f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

4. Add annotations (description) to the new model

In [47]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version, # here we pass in the model_version instead of run_id
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1686149579432, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2023-06-07', last_updated_timestamp=1686149834720, name='nyc-taxi-regressor', run_id='30f8bee56de544a0945d74fb23f3183f', run_link=None, source='/Users/yujingma/Documents/Learning/MLOPS/mlops-zoomcamp/02-experiment-tracking/mlruns/1/30f8bee56de544a0945d74fb23f3183f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [34]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [36]:
df = read_dataframe("data/green_tripdata_2021-03.csv")

/var/folders/42/f9s_rgk15078ym2w50_xtc180000gq/T/ipykernel_5486/3050441246.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [37]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/Users/cristian.martinez/Repositories/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [38]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [39]:
X_test = preprocess(df, dv)

In [40]:
target = "duration"
y_test = df[target].values

In [41]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 139 ms, sys: 44.6 ms, total: 183 ms
Wall time: 447 ms


{'rmse': 6.659623830022514}

In [42]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 6.94 s, sys: 216 ms, total: 7.16 s
Wall time: 7.28 s


{'rmse': 6.881555517147188}

In [43]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1652971637398, current_stage='Production', description='The model version 4 was transitioned to Staging on 2022-05-19', last_updated_timestamp=1652972763255, name='nyc-taxi-regressor', run_id='b8904012c84343b5bf8ee72aa8f0f402', run_link=None, source='./mlruns/1/b8904012c84343b5bf8ee72aa8f0f402/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>